In [130]:
import boto3
import json
import csv as csv
import PIL

from PIL import Image

In [ ]:
# Define the function to call the AWS text detection service. Return the bounding box data of each word detected
def detect_text(photo, bucket):
    client = boto3.client('rekognition')

    response = client.detect_text(Image={'S3Object': {'Bucket': bucket, 'Name': photo}})

    textDetections = response['TextDetections']

    words = {}
    for text in textDetections:
        if (text['Type'] == 'WORD'):
            words[text['DetectedText']] = {'Left': text['Geometry']['BoundingBox']['Left'],
                                           'Top': text['Geometry']['BoundingBox']['Top'],
                                           'Width': text['Geometry']['BoundingBox']['Width'],
                                           'Height': text['Geometry']['BoundingBox']['Height']}
    return words

In [135]:
# Change pname to the jpg file in S3 (only ss1-6 exist atm)
pname = 'ss3'
bucket = 'capstone-bucket-asu-21221'
photo = pname + '.jpg'
word_list = detect_text(photo, bucket)

In [136]:
# Grab the image resolution and the mouse click coordinates
img = PIL.Image.open('H:\\Lab\\Keyword Mapping\\'+ pname + '\\' + photo)
wid, hgt = img.size
screen_res = [wid, hgt]
coord_list = []
with open('H:\\Lab\\Keyword Mapping\\'+ pname + '\\' + pname + '_coords.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if row[0] != 'Click-X':
            coord_list.append([int(row[0]), int(row[1])])

result = {}

In [137]:
# For every mouse click coordinate, try to assign it to the corresponding bounding box
for coords in coord_list:
    for word in word_list:
        left = word_list[word]['Left'] * screen_res[0]
        right = left + (word_list[word]['Width'] * screen_res[0])
        top = word_list[word]['Top'] * screen_res[1]
        bottom = top + (word_list[word]['Height'] * screen_res[1])
        
        '''
        prec = 0.01
        lp = left * prec
        rp = right * prec
        tp = top * prec
        bp = bottom * prec
        '''

        if coords[0] >= left and coords[0] <= right and coords[1] >= top and coords[1] <= bottom:
            if word in result:
                result[word] += 1
            else:
                result[word] = 1
            break

In [138]:
# Print results
for r in result:
    print(str(r) + "," + str(result[r]))

INFORMATION,9
adidas,10
Polyamide.36%,5
Women's,2
*****,3
Shipping.,1
I,1
closure,1
Wash,2
Machine,4
3-Stripes,1
ratings,1
2.0,1
136,1
questions,1
Opaque,1
64%,1
Closure,1
Available,1
